In [5]:
import sys,glob
sys.path.append('/home/simon/git/burns/code/')

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data_dir = '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/'
blank_mz_tol = 5
mz_tol = 5

In [8]:
group1 = ['MSS1','MSS2','MSS3','MSS4','MSS5','MSS6','MSS13']
group2 = ['MSS10','MSS11','MSS14']
files1 = []
files2 = []
for g in group1:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files1.append((match,data_dir+bfilename))
for g in group2:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files2.append((match,data_dir + bfilename))

group1,_ = zip(*files1)
group2,_ = zip(*files2)

print len(group1),len(group2)
print group1
print group2

12 9
('/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS1a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS2a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS3a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS4a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a3.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a2.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS6a.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a1.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a2.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a3.mzML', '/home/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a.mzML')
('/home/simon/Dropbox/BioResearch/Met

In [9]:
from load_di import load_di

# s = load_di(group1[0])
import glob
data = {}
for f in files1+files2:
    out_file = '../csv_files/' + f[0].split('/')[-1].split('.')[0]+'.csv'
#     data[f[0]] = load_di(f[0],blank_file = f[1],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)
    data[f[0]] = load_di(f[0],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)


In [38]:
normalised_data = {}
for f,spec in data.items():
    mz_list,i_list = zip(*spec)
    tot_intensity = sum(i_list)
    print tot_intensity
    new_intensity = [1000.0*i/tot_intensity for i in i_list]
    new_spec = zip(mz_list,new_intensity)
    normalised_data[f] = new_spec

324412649.465
436240818.691
380777529.477
288935937.354
400955613.605
806308809.869
762854905.605
301898846.857
289888093.164
306846745.561
464233290.262
292485624.916
2107366630.76
320459909.518
450357167.426
277990991.938
307305363.902
476529778.904
506194463.586
462673639.387
290878928.832
414260544.383


Do some of Greedy matching

In [11]:
import numpy as np
from matching import Greedy
g = Greedy()
mp= g.process(data,mz_tol)
from scipy.stats import ttest_ind
small = 0.001
test_results = []
for ps in mp:
    g1 = []
    g2 = []
    for p in ps.peaks:
        filename = p[2]
        if filename in group1:
            g1.append(p[1])
        else:
            g2.append(p[1])
    if len(g1) <2 and len(g2) < 2:
        continue
    else:
        for i in range(len(g1),len(group1)):
            g1.append(small)
        for i in range(len(g2),len(group2)):
            g2.append(small)
        for i,g in enumerate(g1):
            if g < small:
                g1[i] = small
#             g1[i] = np.log(g1[i])
        for i,g in enumerate(g2):
            if g < small:
                g2[i] = small
#             g2[i] = np.log(g2[i])


        t,p = ttest_ind(g1,g2,equal_var=True)
        test_results.append((ps,t,p,g1,g2))
        
test_results = sorted(test_results,key = lambda x: x[2])
for ps,t,p,g1,g2 in test_results[:]:
#     if np.floor(ps.mean_mz) == 84:
        print ps.mean_mz,p
#         g1peaks = []
#         g2peaks = []
#         for p in ps.peaks:
#             if p[2] in group1:
#                 g1peaks.append(p)
#             else:
#                 g2peaks.append(p)
#         print "Group1:"
#         for p in g1peaks:
#             print '\t',p[1],p[2]
#         print
#         print "Group2:"
#         for p in g2peaks:
#             print '\t',p[1],p[2]
#         print
        print
        for g in g1:
            print g
        print
        for g in g2:
            print g
        print
        print
        print
        print
        print
        

9858 peaks
279.159218521 1.5402352575e-05

1906508.0
1891911.375
1697109.25
1666368.625
1629545.75
1444555.875
1781327.375
1539271.125
2294362.0
1826642.25
1302106.125
1144857.625

1198114.0
994032.3125
931715.1875
739966.625
738229.875
539306.125
1142264.625
1122390.5
0.001





330.336942365 3.58037274971e-05

166504.8125
289085.6875
328971.15625
174022.265625
214196.40625
338783.09375
115149.515625
333131.84375
229202.21875
308182.625
180086.46875
110471.8125

126920.664062
119431.742188
131096.765625
90422.96875
0.001
0.001
0.001
0.001
0.001





210.170144183 6.85001728097e-05

251479.21875
366620.625
185896.265625
244350.984375
462090.96875
299717.4375
300844.59375
430789.21875
233418.375
187528.234375
225634.09375
185580.8125

161959.1875
136849.25
140979.9375
223890.765625
0.001
0.001
0.001
0.001
0.001





227.164318293 0.000143057623566

231681.125
273974.875
371714.75
211360.296875
257689.359375
175245.375
184330.15625
236066.78125
219220.5625
231344.890625
183260.515625
0.0

311.065191286 0.0390678916739

290502.9375
47290.7539062
254327.546875
779088.0
464870.5625
186234.9375
630698.75
0.001
0.001
0.001
0.001
0.001

1783326.875
1809600.625
1618768.75
1123113.5
310627.9375
201027.03125
75984.453125
0.001
0.001





264.038392814 0.03914545611

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

81784.8359375
121862.164062
65095.3359375
0.001
0.001
0.001
0.001
0.001
0.001





314.932200736 0.0393279497708

139262.40625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

159490.609375
151897.953125
206969.90625
185406.75
0.001
0.001
0.001
0.001
0.001





245.000029596 0.0394977820233

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

408465.25
383969.34375
205802.0
0.001
0.001
0.001
0.001
0.001
0.001





145.122511191 0.0395788162031

2895113.25
3551142.0
2454832.75
2239217.75
3421458.25
3227575.25
3582666.0
3155911.75
3204612.75
3101006.0
3538112.75
3908462.5

2401062.25
2015071.5
1894184.625
139

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

194437.578125
230316.578125
340109.1875
0.001
0.001
0.001
0.001
0.001
0.001





296.962966365 0.115148693595

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

53516.796875
105634.9375
0.001
0.001
0.001
0.001
0.001
0.001
0.001





529.075571803 0.115373092456

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

120420.125
60764.5703125
0.001
0.001
0.001
0.001
0.001
0.001
0.001





233.021334146 0.1155679148

309897.9375
143484.84375
523458.5625
702957.875
1723564.125
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





185.153848779 0.117055974927

291131.09375
283253.8125
253962.84375
317156.21875
262277.03125
290586.6875
308200.71875
259614.734375
287511.0625
286807.28125
0.001
0.001

246359.703125
292625.0
310217.34375
205755.015625
258871.3125
0.001
0.001
0.001
0.001





167.105770849 0.11705984624

259871.46875
241547.5

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





315.004115395 0.2175314548

260615.734375
277164.375
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





433.092912104 0.217593386557

173717.046875
162174.375
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





71.95212876 0.217594966671

386499.0625
248175.546875
139966.015625
1406691.0
492726.375
352163.65625
480841.65625
411651.6875
343407.84375
351609.0
794890.75
671238.8125

476693.03125
561569.375
480649.5
513846.8125
526248.3125
493434.875
960989.1875
1025627.375
11394235.0





275.277495258 0.217671910071

30466.1953125
32902.0742188
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





120.024092106 0.217681857616

211307.4375
228424.4375
0.001
0.001
0.001
0.001
0.001
0.

170481.5625
666449.875
621582.1875
655961.3125
1148405.25
453016.21875
0.001





365.2777808 0.299513079277

182579.953125
729537.5
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





258.101552166 0.299585264395

151690.515625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

137714.265625
286041.34375
0.001
0.001
0.001
0.001
0.001
0.001
0.001





198.05281982 0.300118668171

141311.8125
688933.9375
1077225.875
2375672.0
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

87956.0703125
353531.59375
398530.03125
0.001
0.001
0.001
0.001
0.001
0.001





544.363671269 0.300281059338

149397.765625
38733.8203125
136360.03125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

57850.8515625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





262.055874401 0.300654303668

402424.9375
99200.5703125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

519495.5
710812.125
785537.9375
921410.4375
669989.8125
208574.234375
0.001





193.002111761 0.52732693171

181966.8125
430340.8125
41951.8828125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

323511.84375
161291.171875
246656.921875
85159.265625
0.001
0.001
0.001
0.001
0.001





339.102971954 0.531150125071

35295.8007812
92041.03125
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

38374.9179688
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





146.09262598 0.531326088154

271831.65625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

198486.8125
219361.78125
0.001
0.001
0.001
0.001
0.001
0.001
0.001





237.028227866 0.533229306777

177653.953125
582855.375
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

1401557.5
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001





121.958145655 0.538430395564

157762.03125
86494.5625
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001

435939.25
0.001
0.001
0.001
0.001
0.001
0.001
0.0

Write out a csv file for processing

In [85]:
for log in ['log','nolog']:
    for missing in ['nan',0.01]:
        for normalise in ['none','tus']:
            options = {'log':log,'missing':missing,'normalise':normalise}
            n_tests = 0

            if options['normalise'] == 'tus':
                normalised_data = {}
                for f,spec in data.items():
                    mz_list,i_list = zip(*spec)
                    tot_intensity = sum(i_list)
                    new_intensity = [1000.0*i/tot_intensity for i in i_list]
                    new_spec = zip(mz_list,new_intensity)
                    normalised_data[f] = new_spec
            elif options['normalise'] == 'none':
                normalised_data = data
            
            g = Greedy()
            mp = g.process(normalised_data,mz_tol)
                
            display_names = {}
            for g in group1+group2:
                display_names[g] = g.split('/')[-1]
            import csv
            all_names = list(group1+group2)
            res = []


            heads = ["mz"] + [display_names[a] for a in all_names] + ["pval"] +["qval"]
            for ps in mp:
                row = [ps.mean_mz]
                g1 = []
                g2 = []
                for n in all_names:
                    peak = filter(lambda x: x[2] == n,ps.peaks)
                    if len(peak) == 0:
                        row.append("nan")
                    else:
                        row.append(peak[0][1])
                    if n in group1:
                        g1.append(row[-1])
                    if n in group2:
                        g2.append(row[-1])
                if options['missing'] == 'nan':
                    g1 = [a for a in g1 if not a == "nan"]
                    g2 = [a for a in g2 if not a == "nan"]
                else:
                    g1 = [a if not a == "nan" else options["missing"] for a in g1]
                    g2 = [a if not a == "nan" else options["missing"] for a in g2]
                if options['log'] == 'log':
                    g1 = [np.log10(a) for a in g1]
                    g2 = [np.log10(a) for a in g2]
                if len(g1) > 1 and len(g2) > 1:
                    t,p = ttest_ind(g1,g2,equal_var=True)
                    n_tests += 1
                else:
                    p = 999
                row.append(p)
                res.append(row)

            res = sorted(res,key = lambda x: x[-1])
            # compute q-value
            for i,r in enumerate(res):
                if r[-1] < 999:
                    q = r[-1]*n_tests / (1.0*(i+1))
                    r.append(q)
                else:
                    r.append("nan")

            with open("spreadsheets/burns_{}_{}_{}.csv".format(options['log'],options['missing'],options['normalise']),'w') as f:
                writer = csv.writer(f)
                writer.writerow(heads)
                for r in res:
                    writer.writerow(r)

9858 peaks
1.30783799326e-05 508 0
1.49641168208e-05 508 1
2.09064738746e-05 508 2
4.66529210589e-05 508 3
0.000127575825227 508 4
0.000149795281785 508 5
0.000483083179896 508 6
0.000484386498584 508 7
0.000520646543644 508 8
0.000692347255274 508 9
0.00073322710209 508 10
0.00110511285985 508 11
0.00111401688952 508 12
0.00125605579019 508 13
0.00161665827413 508 14
0.00185992840488 508 15
0.00226473298337 508 16
0.0022671842212 508 17
0.00245024555956 508 18
0.00262540179084 508 19
0.00272577539397 508 20
0.00311427329757 508 21
0.00313225951923 508 22
0.00325511641006 508 23
0.00345491682824 508 24
0.00367324315009 508 25
0.00409483237726 508 26
0.00432984657532 508 27
0.00531343682525 508 28
0.00574651788794 508 29
0.0059354169864 508 30
0.00659593756198 508 31
0.0068411959965 508 32
0.00687031131202 508 33
0.00688046516812 508 34
0.00696965778568 508 35
0.00705016791865 508 36
0.00779467572925 508 37
0.00814886620479 508 38
0.00839593135013 508 39
0.00843799673968 508 40
0.008858

0.476987245617 508 329
0.477054234557 508 330
0.486790737099 508 331
0.491472312755 508 332
0.491744650393 508 333
0.492682072459 508 334
0.495864547672 508 335
0.50260671032 508 336
0.502937591502 508 337
0.507646173515 508 338
0.508949198469 508 339
0.51588474115 508 340
0.516682672398 508 341
0.520628136531 508 342
0.520964239714 508 343
0.523362409978 508 344
0.523537823432 508 345
0.52431315435 508 346
0.524983700398 508 347
0.528146592752 508 348
0.528725917485 508 349
0.532908408343 508 350
0.533825158228 508 351
0.535593187459 508 352
0.549791975532 508 353
0.550912329962 508 354
0.551178517807 508 355
0.552936355559 508 356
0.553836480338 508 357
0.556872090043 508 358
0.560137580577 508 359
0.565018809986 508 360
0.566094938912 508 361
0.567473360926 508 362
0.567604135854 508 363
0.567782013804 508 364
0.569146418574 508 365
0.572154731316 508 366
0.574729777222 508 367
0.575706204116 508 368
0.579117197772 508 369
0.584354663317 508 370
0.587225191958 508 371
0.589201465785

0.390173612046 2670 1426
0.390301068164 2670 1427
0.390325084137 2670 1428
0.390692702383 2670 1429
0.392428930445 2670 1430
0.392911933218 2670 1431
0.39293751126 2670 1432
0.393578982983 2670 1433
0.393786279691 2670 1434
0.394289420775 2670 1435
0.395845114331 2670 1436
0.395845244414 2670 1437
0.396315660331 2670 1438
0.397056954773 2670 1439
0.39817907686 2670 1440
0.398748401058 2670 1441
0.399511951334 2670 1442
0.399576428095 2670 1443
0.399587426943 2670 1444
0.399803235026 2670 1445
0.400304622382 2670 1446
0.400304622382 2670 1447
0.400304622382 2670 1448
0.400304622382 2670 1449
0.400304622382 2670 1450
0.400304622382 2670 1451
0.400304622382 2670 1452
0.400304622382 2670 1453
0.400304622382 2670 1454
0.400304622382 2670 1455
0.400304622382 2670 1456
0.400304622382 2670 1457
0.400304622382 2670 1458
0.400304622382 2670 1459
0.400304622382 2670 1460
0.400304622382 2670 1461
0.400304622382 2670 1462
0.400304622382 2670 1463
0.400304622382 2670 1464
0.400304622382 2670 1465
0.

0.782642737954 2670 2475
0.782794970414 2670 2476
0.784137383243 2670 2477
0.785137613587 2670 2478
0.786202283437 2670 2479
0.786782937036 2670 2480
0.786932955891 2670 2481
0.787039278414 2670 2482
0.78758809686 2670 2483
0.78850917599 2670 2484
0.78920159449 2670 2485
0.789303469791 2670 2486
0.78975433104 2670 2487
0.790706569031 2670 2488
0.791159375992 2670 2489
0.792418559105 2670 2490
0.792584599912 2670 2491
0.793018107157 2670 2492
0.795130866444 2670 2493
0.795149519092 2670 2494
0.795188902728 2670 2495
0.797301857111 2670 2496
0.797327832757 2670 2497
0.799404232497 2670 2498
0.800047338177 2670 2499
0.800156048675 2670 2500
0.803160994994 2670 2501
0.803632984094 2670 2502
0.805531928717 2670 2503
0.80653612674 2670 2504
0.80692387599 2670 2505
0.806957817083 2670 2506
0.807230309458 2670 2507
0.807598448732 2670 2508
0.808141924858 2670 2509
0.811191301113 2670 2510
0.81282363663 2670 2511
0.816643913604 2670 2512
0.816908126415 2670 2513
0.817339574964 2670 2514
0.81790

0.400304622382 2670 1713
0.400304622382 2670 1714
0.400304622382 2670 1715
0.400304622382 2670 1716
0.400304622382 2670 1717
0.400304622382 2670 1718
0.400304622382 2670 1719
0.400304622382 2670 1720
0.400304622382 2670 1721
0.400304622382 2670 1722
0.400304622382 2670 1723
0.400304622382 2670 1724
0.400304622382 2670 1725
0.400304622382 2670 1726
0.400304622382 2670 1727
0.400304622382 2670 1728
0.400304622382 2670 1729
0.400304622382 2670 1730
0.400304622382 2670 1731
0.400304622382 2670 1732
0.400304622382 2670 1733
0.400304622382 2670 1734
0.400304622382 2670 1735
0.400304622382 2670 1736
0.400304622382 2670 1737
0.400304622382 2670 1738
0.400304622382 2670 1739
0.400304622382 2670 1740
0.400304622382 2670 1741
0.400304622382 2670 1742
0.400304622382 2670 1743
0.400304622382 2670 1744
0.400304622382 2670 1745
0.400304622382 2670 1746
0.400304622382 2670 1747
0.400304622382 2670 1748
0.400304622382 2670 1749
0.400304622382 2670 1750
0.400304622382 2670 1751
0.400304622382 2670 1752


...

0.461611564273 508 323
0.461730016967 508 324
0.469488454709 508 325
0.472232735516 508 326
0.475901552478 508 327
0.478959472418 508 328
0.478997264204 508 329
0.480186095429 508 330
0.482063306269 508 331
0.482077829179 508 332
0.484163102759 508 333
0.485319155272 508 334
0.487119227018 508 335
0.487562731171 508 336
0.488141374322 508 337
0.488731519337 508 338
0.490004346516 508 339
0.492038066247 508 340
0.494266522901 508 341
0.496602219099 508 342
0.497414856873 508 343
0.497624548062 508 344
0.5020515328 508 345
0.50231877627 508 346
0.502686828037 508 347
0.505276741464 508 348
0.508894847447 508 349
0.509508389549 508 350
0.513034991373 508 351
0.513261548911 508 352
0.520594810058 508 353
0.522786957979 508 354
0.527100663788 508 355
0.528179831964 508 356
0.533286835572 508 357
0.537065929236 508 358
0.540921779364 508 359
0.546846379963 508 360
0.548134771311 508 361
0.552696418881 508 362
0.555144820661 508 363
0.555941356345 508 364
0.557277825405 508 365
0.558583984615

0.553087505489 508 344
0.554406829036 508 345
0.555186670155 508 346
0.555256284418 508 347
0.557993374657 508 348
0.558002844693 508 349
0.566055029498 508 350
0.568272064945 508 351
0.57015396594 508 352
0.570682301623 508 353
0.571137576496 508 354
0.572382961838 508 355
0.578981142379 508 356
0.581496788276 508 357
0.582496497995 508 358
0.589311743056 508 359
0.59064275509 508 360
0.591295861507 508 361
0.593480591281 508 362
0.596359139667 508 363
0.605172046419 508 364
0.607878328637 508 365
0.611594828264 508 366
0.612995514722 508 367
0.613654733801 508 368
0.614539231705 508 369
0.617593407412 508 370
0.619542946327 508 371
0.620786443141 508 372
0.622106962433 508 373
0.623699982622 508 374
0.62542606534 508 375
0.627233036618 508 376
0.627530861159 508 377
0.629135340236 508 378
0.630718200063 508 379
0.634448435798 508 380
0.639282513066 508 381
0.639494395148 508 382
0.648177875662 508 383
0.65454205168 508 384
0.658695544549 508 385
0.661198185495 508 386
0.661395157263 

0.338085267566 2670 1420
0.338601313306 2670 1421
0.338625273366 2670 1422
0.339009264097 2670 1423
0.340694556943 2670 1424
0.340773670367 2670 1425
0.341111521652 2670 1426
0.341782756669 2670 1427
0.341895682695 2670 1428
0.342079784751 2670 1429
0.342442421518 2670 1430
0.342734356727 2670 1431
0.34356200726 2670 1432
0.344289296913 2670 1433
0.345071074912 2670 1434
0.345152713346 2670 1435
0.345482099177 2670 1436
0.345689512586 2670 1437
0.346660147477 2670 1438
0.348901544254 2670 1439
0.349649359913 2670 1440
0.349842757457 2670 1441
0.349892957232 2670 1442
0.349906722711 2670 1443
0.349962044629 2670 1444
0.350236804462 2670 1445
0.350386303215 2670 1446
0.351090756774 2670 1447
0.351683130243 2670 1448
0.352696825165 2670 1449
0.352831210074 2670 1450
0.353711884395 2670 1451
0.354013626393 2670 1452
0.356508071889 2670 1453
0.357145878742 2670 1454
0.357298927775 2670 1455
0.358466246128 2670 1456
0.358655411526 2670 1457
0.358949490878 2670 1458
0.360007024262 2670 1459
0

0.656371211219 2670 2467
0.657632403712 2670 2468
0.658470555196 2670 2469
0.666398890724 2670 2470
0.670503250267 2670 2471
0.671617235995 2670 2472
0.673000242663 2670 2473
0.673943829552 2670 2474
0.674825020529 2670 2475
0.681452547091 2670 2476
0.684294845395 2670 2477
0.685931288578 2670 2478
0.689049351166 2670 2479
0.691078462273 2670 2480
0.691174325431 2670 2481
0.693737150659 2670 2482
0.696480368056 2670 2483
0.698291268869 2670 2484
0.702789175657 2670 2485
0.704624900499 2670 2486
0.705344054565 2670 2487
0.705522567878 2670 2488
0.705888208211 2670 2489
0.706048557327 2670 2490
0.706059094042 2670 2491
0.706864088196 2670 2492
0.708445231025 2670 2493
0.708489054561 2670 2494
0.709181551936 2670 2495
0.709185922878 2670 2496
0.710302371893 2670 2497
0.711345560569 2670 2498
0.714920048747 2670 2499
0.716894378032 2670 2500
0.722039947041 2670 2501
0.726446203058 2670 2502
0.730907770106 2670 2503
0.732416343225 2670 2504
0.732980784248 2670 2505
0.736945815757 2670 2506


0.400304622382 2670 1698
0.400304622382 2670 1699
0.400304622382 2670 1700
0.400304622382 2670 1701
0.400304622382 2670 1702
0.400304622382 2670 1703
0.400304622382 2670 1704
0.400304622382 2670 1705
0.400304622382 2670 1706
0.400304622382 2670 1707
0.400304622382 2670 1708
0.400304622382 2670 1709
0.400304622382 2670 1710
0.400304622382 2670 1711
0.400304622382 2670 1712
0.400304622382 2670 1713
0.400304622382 2670 1714
0.400304622382 2670 1715
0.400304622382 2670 1716
0.400304622382 2670 1717
0.400304622382 2670 1718
0.400304622382 2670 1719
0.400304622382 2670 1720
0.400304622382 2670 1721
0.400304622382 2670 1722
0.400304622382 2670 1723
0.400304622382 2670 1724
0.400304622382 2670 1725
0.400304622382 2670 1726
0.400304622382 2670 1727
0.400304622382 2670 1728
0.400304622382 2670 1729
0.400304622382 2670 1730
0.400304622382 2670 1731
0.400304622382 2670 1732
0.400304622382 2670 1733
0.400304622382 2670 1734
0.400304622382 2670 1735
0.400304622382 2670 1736
0.400304622382 2670 1737


In [71]:
for r in res[:10]:
    print r[0],r[-1]

279.159218521 1.35563390372e-05
462.146843969 1.73407167063e-05
305.251726956 2.9348596183e-05
304.248307105 5.55628498956e-05
356.279807222 0.000268902663636
280.162638745 0.000361379530556
295.070290712 0.000570483562544
158.154128944 0.000677048047452
143.106848706 0.000924332932252
288.225168757 0.00115745746591
